In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 73)

In [3]:
team_df = pd.read_pickle('data/team_stats_scraped.pickle')
game_df = pd.read_pickle('data/games_scraped.pickle')

In [4]:
game_df.shape

(29399, 26)

In [5]:
mask = (game_df.team == 'chi') & (game_df.year == 2019)
game_df[mask]

,team,year,opp,week_num,game_day_of_week,game_date,game_outcome,team_record,game_location,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st
4819,chi,2019,gnb,1,Thu,September 5,L,0-1,NaN,3.0,10.0,16.0,254.0,208.0,46.0,1.0,13.0,213.0,166.0,47.0,NaN,8.0,NaN,-20.0,9.0,-2.0
4820,chi,2019,den,2,Sun,September 15,W,1-1,@,16.0,14.0,18.0,273.0,120.0,153.0,NaN,27.0,372.0,282.0,90.0,1.0,4.0,NaN,4.0,-4.0,5.0
4821,chi,2019,was,3,Mon,September 23,W,2-1,@,31.0,15.0,21.0,298.0,208.0,90.0,1.0,25.0,356.0,287.0,69.0,5.0,8.0,NaN,9.0,11.0,-3.0
4822,chi,2019,min,4,Sun,September 29,W,3-1,NaN,16.0,6.0,17.0,269.0,197.0,72.0,NaN,15.0,222.0,182.0,40.0,2.0,4.0,NaN,-2.0,10.0,1.0
4823,chi,2019,rai,5,Sun,October 6,L,3-2,@,21.0,24.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,1.0,NaN,-3.0,-2.0,-0.0
4824,chi,2019,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4825,chi,2019,nor,7,Sun,October 20,L,3-3,NaN,25.0,36.0,16.0,252.0,235.0,17.0,2.0,24.0,424.0,273.0,151.0,NaN,4.0,NaN,-12.0,-13.0,10.0
4826,chi,2019,sdg,8,Sun,October 27,L,3-4,NaN,16.0,17.0,26.0,388.0,226.0,162.0,2.0,11.0,231.0,195.0,36.0,1.0,1.0,NaN,0.0,5.0,-6.0
4827,chi,2019,phi,9,Sun,November 3,L,3-5,@,14.0,22.0,10.0,164.0,102.0,62.0,1.0,26.0,373.0,227.0,146.0,NaN,1.0,NaN,-4.0,-10.0,2.0
4828,chi,2019,det,10,Sun,November 10,W,4-5,NaN,20.0,13.0,13.0,226.0,145.0,81.0,NaN,21.0,357.0,259.0,98.0,1.0,1.0,NaN,-0.0,6.0,-2.0


In [6]:
# clean the game_location column & apply change. def home_game(row):
def home_game(row):    
    if row == '@':
        return 0
    return 1

game_df['home'] = game_df.game_location.apply(home_game)
game_df.drop("game_location", axis=1, errors='ignore', inplace=True)

In [7]:
# add team_year identifier to each row (to easily groupby team/year)

team_years = game_df['team'] + '-' +  game_df['year'].astype(str) 
game_df.insert(loc=2, column='team_year', value=team_years)
game_df.sample()

,team,year,team_year,opp,week_num,game_day_of_week,game_date,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home
24655,sfo,2014,sfo-2014,was,12,Sun,November 23,W,7-4,17.0,13.0,16.0,312.0,246.0,66.0,3.0,12.0,213.0,77.0,136.0,1.0,4.0,NaN,-3.0,12.0,-1.0,1


Datetime formatting strings:

- ``%B`` month fullname
- ``%-d`` day of month without leading zero\*
    - \*note: the ``-`` only works on Linux/OS-X...for windows replace with '#'
- ``%Y`` full year

In [8]:
# convert game date to datetime & add new column to DF


f = r"%B %d-%Y"
full_game_date = game_df.game_date + '-' + game_df.year.astype(str)
full_game_date = pd.to_datetime(full_game_date, format=f)
game_df.insert(loc=2, column='date', value=full_game_date)
game_df.sample(3)

,team,year,date,team_year,opp,week_num,game_day_of_week,game_date,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home
22159,pit,1991,1991-10-27,pit-1991,cle,9,Sun,October 27,L,3-5,14.0,17.0,18.0,282.0,207.0,75.0,2.0,19.0,262.0,179.0,83.0,NaN,4.0,NaN,NaN,NaN,NaN,0
19035,nyj,1988,1988-11-27,nyj-1988,mia,13,Sun,November 27,W,6-6-1,38.0,34.0,39.0,597.0,426.0,171.0,2.0,20.0,440.0,353.0,87.0,1.0,1.0,NaN,NaN,NaN,NaN,1
23852,sfo,1969,1969-09-21,sfo-1969,atl,1,Sun,September 21,L,0-1,12.0,24.0,19.0,331.0,200.0,131.0,5.0,13.0,308.0,79.0,229.0,2.0,NaN,NaN,NaN,NaN,NaN,0


In [9]:
# now drop the game_date col
game_df.drop("game_date", axis=1, inplace=True, errors='ignore')
game_df.sample(2) # to confirm

,team,year,date,team_year,opp,week_num,game_day_of_week,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home
5925,cle,1972,1972-10-29,cle-1972,den,7,Sun,W,4-3,27.0,20.0,26.0,343.0,171.0,172.0,1.0,17.0,272.0,126.0,146.0,1.0,4.0,NaN,NaN,NaN,NaN,0
5829,cle,1966,1966-10-08,cle-1966,pit,5,Sat,W,3-2,41.0,10.0,31.0,467.0,226.0,241.0,2.0,12.0,204.0,182.0,22.0,6.0,NaN,NaN,NaN,NaN,NaN,1


In [10]:
# add decade column
decades = game_df['year'] // 10
game_df.insert(loc=5, column='decade', value=decades)
game_df[['date', 'decade']].sample(10)

,date,decade
3691,NaT,201
9488,2005-10-09,200
20180,1994-12-18,199
4287,1988-01-08,198
8998,1975-12-21,197
1107,1974-12-07,197
6504,2010-10-17,201
9401,2000-09-17,200
25652,1966-11-13,196
6823,1969-10-12,196


In [11]:
# add turnovers^2 col

game_df[['to2_off', 'to2_def']] = game_df[['to_off', 'to_def']].to_numpy() ** 2
game_df[['to_off', 'to2_off', 'to_def', 'to2_def']]

,to_off,to2_off,to_def,to2_def
0,5.0,25.0,6.0,36.0
1,5.0,25.0,6.0,36.0
2,6.0,36.0,3.0,9.0
3,6.0,36.0,NaN,NaN
4,3.0,9.0,2.0,4.0
...,...,...,...,...
29394,2.0,4.0,1.0,1.0
29395,4.0,16.0,2.0,4.0
29396,2.0,4.0,3.0,9.0
29397,NaN,NaN,NaN,NaN


In [12]:
# add unique game identifier that will be the same for both teams in the same game


def apply_game_id(row):
    teams = []
    teams.append(str(row['team']))
    teams.append(str(row['opp']))
    teams.sort()
    
    game_id = teams[0] + '-' +  teams[1] + '-' + str(row['date'])[:-9]
    return game_id

In [13]:
game_df['game_id'] = game_df.apply(apply_game_id, axis=1)
game_df

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,to2_off,to2_def,game_id
0,crd,1960,1960-09-23,crd-1960,ram,196,1,Fri,W,1-0,43.0,21.0,20.0,430.0,281.0,149.0,5.0,16.0,238.0,188.0,50.0,6.0,NaN,NaN,NaN,NaN,NaN,0,25.0,36.0,crd-ram-1960-09-23
1,crd,1960,1960-10-02,crd-1960,nyg,196,2,Sun,L,1-1,14.0,35.0,11.0,222.0,115.0,107.0,5.0,18.0,405.0,268.0,137.0,6.0,NaN,NaN,NaN,NaN,NaN,1,25.0,36.0,crd-nyg-1960-10-02
2,crd,1960,1960-10-09,crd-1960,phi,196,3,Sun,L,1-2,27.0,31.0,20.0,364.0,145.0,219.0,6.0,14.0,293.0,199.0,94.0,3.0,NaN,NaN,NaN,NaN,NaN,0,36.0,9.0,crd-phi-1960-10-09
3,crd,1960,1960-10-16,crd-1960,pit,196,4,Sun,L,1-3,14.0,27.0,20.0,295.0,154.0,141.0,6.0,12.0,237.0,146.0,91.0,NaN,NaN,NaN,NaN,NaN,NaN,0,36.0,NaN,crd-pit-1960-10-16
4,crd,1960,1960-10-23,crd-1960,dal,196,5,Sun,W,2-3,12.0,10.0,17.0,306.0,80.0,226.0,3.0,9.0,175.0,124.0,51.0,2.0,NaN,NaN,NaN,NaN,NaN,1,9.0,4.0,crd-dal-1960-10-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29394,was,2020,2020-12-20,was-2020,sea,202,15,Sun,L,6-8,15.0,20.0,26.0,353.0,269.0,84.0,2.0,16.0,302.0,121.0,181.0,1.0,1.0,NaN,1.0,-3.0,-4.0,1,4.0,1.0,sea-was-2020-12-20
29395,was,2020,2020-12-27,was-2020,car,202,16,Sun,L,6-9,13.0,20.0,20.0,386.0,278.0,108.0,4.0,19.0,280.0,167.0,113.0,2.0,4.0,NaN,-5.0,5.0,-7.0,1,16.0,4.0,car-was-2020-12-27
29396,was,2020,2020-01-03,was-2020,phi,202,17,Sun,W,7-9,20.0,14.0,16.0,248.0,145.0,103.0,2.0,16.0,216.0,98.0,118.0,3.0,8.0,NaN,-8.0,11.0,3.0,0,4.0,9.0,phi-was-2020-01-03
29397,was,2020,NaT,was-2020,NaN,202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,nan-was-


In [14]:
cat_game_outcome = game_df.loc[:, ['game_outcome']]
cat_game_outcome.value_counts()

game_outcome
L               13773
W               13773
T                 254
dtype: int64

In [15]:
# convert game_outcome col to binary to avoid multicolinearity

game_df[['result_tie', 'result_win']] = pd.get_dummies(game_df.game_outcome, drop_first=True)
game_df.sample(10)

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,to2_off,to2_def,game_id,result_tie,result_win
22282,pit,1997,1997-12-07,pit-1997,den,199,15,Sun,W,10-4,35.0,24.0,22.0,476.0,290.0,186.0,2.0,18.0,320.0,231.0,89.0,1.0,1.0,NaN,19.0,-2.0,-1.0,1,4.0,1.0,den-pit-1997-12-07,0,1
26759,tam,1986,1986-12-14,tam-1986,gnb,198,15,Sun,L,2-13,7.0,21.0,14.0,242.0,117.0,125.0,2.0,20.0,349.0,173.0,176.0,NaN,1.0,NaN,NaN,NaN,NaN,1,4.0,NaN,gnb-tam-1986-12-14,0,0
21642,phi,2019,2019-12-15,phi-2019,was,201,15,Sun,W,7-7,37.0,27.0,27.0,415.0,258.0,157.0,1.0,19.0,352.0,251.0,101.0,1.0,1.0,NaN,19.0,-5.0,-3.0,0,1.0,1.0,phi-was-2019-12-15,0,1
8854,det,1965,1965-12-05,det-1965,sfo,196,12,Sun,L,5-6-1,14.0,17.0,16.0,300.0,155.0,145.0,1.0,20.0,357.0,209.0,148.0,1.0,NaN,NaN,NaN,NaN,NaN,0,1.0,1.0,det-sfo-1965-12-05,0,0
14007,mia,1987,1987-10-04,mia-1987,sea,198,4,Sun,L,1-2,20.0,24.0,19.0,324.0,160.0,164.0,2.0,20.0,402.0,326.0,76.0,3.0,4.0,NaN,NaN,NaN,NaN,0,4.0,9.0,mia-sea-1987-10-04,0,0
27833,oti,1990,1990-12-16,oti-1990,kan,199,15,Sun,W,8-6,27.0,10.0,22.0,563.0,505.0,58.0,1.0,17.0,350.0,249.0,101.0,2.0,1.0,NaN,NaN,NaN,NaN,0,1.0,4.0,kan-oti-1990-12-16,0,1
6574,cle,2014,2014-10-26,cle-2014,rai,201,8,Sun,W,4-3,23.0,13.0,15.0,306.0,267.0,39.0,NaN,19.0,387.0,311.0,76.0,3.0,4.0,NaN,-0.0,11.0,0.0,1,NaN,9.0,cle-rai-2014-10-26,0,1
8168,den,1987,1987-01-10,den-1987,oti,198,Division,Sun,W,11-4-1,34.0,10.0,19.0,316.0,255.0,61.0,1.0,20.0,337.0,264.0,73.0,3.0,4.0,NaN,NaN,NaN,NaN,1,1.0,9.0,den-oti-1987-01-10,0,1
19775,rai,1971,1971-10-10,rai-1971,den,197,4,Sun,W,3-1,27.0,16.0,20.0,269.0,113.0,156.0,1.0,11.0,319.0,220.0,99.0,3.0,4.0,NaN,NaN,NaN,NaN,0,1.0,9.0,den-rai-1971-10-10,0,1
10051,gnb,1979,1979-10-07,gnb-1979,atl,197,6,Sun,L,2-4,7.0,25.0,17.0,275.0,158.0,117.0,3.0,19.0,286.0,134.0,152.0,1.0,1.0,NaN,NaN,NaN,NaN,0,9.0,1.0,atl-gnb-1979-10-07,0,0


In [16]:
mask = game_df.game_outcome == "T"
game_df[mask].head()

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,to2_off,to2_def,game_id,result_tie,result_win
9,crd,1960,1960-11-27,crd-1960,cle,196,10,Sun,T,5-4-1,17.0,17.0,21.0,322.0,127.0,195.0,6.0,17.0,274.0,88.0,186.0,2.0,NaN,NaN,NaN,NaN,NaN,1,36.0,4.0,cle-crd-1960-11-27,1,0
30,crd,1962,1962-10-14,crd-1962,was,196,5,Sun,T,1-3-1,17.0,17.0,16.0,309.0,251.0,58.0,2.0,15.0,321.0,259.0,62.0,1.0,NaN,NaN,NaN,NaN,NaN,1,4.0,1.0,crd-was-1962-10-14,1,0
55,crd,1964,1964-09-20,crd-1964,cle,196,2,Sun,T,1-0-1,33.0,33.0,17.0,344.0,230.0,114.0,3.0,18.0,281.0,166.0,115.0,3.0,NaN,NaN,NaN,NaN,NaN,0,9.0,9.0,cle-crd-1964-09-20,1,0
63,crd,1964,1964-11-15,crd-1964,nyg,196,10,Sun,T,5-3-2,10.0,10.0,12.0,143.0,105.0,38.0,3.0,12.0,212.0,131.0,81.0,3.0,NaN,NaN,NaN,NaN,NaN,1,9.0,9.0,crd-nyg-1964-11-15,1,0
87,crd,1966,1966-10-16,crd-1966,dal,196,6,Sun,T,5-0-1,10.0,10.0,13.0,175.0,130.0,45.0,NaN,17.0,326.0,191.0,135.0,4.0,NaN,NaN,NaN,NaN,NaN,1,NaN,16.0,crd-dal-1966-10-16,1,0


In [17]:


game_df = (game_df.assign(wins=game_df.team_record.str.split('-').str.get(0),
                          losses=game_df.team_record.str.split('-').str.get(1),
                          ties=game_df.team_record.str.split('-').str.get(2)))
game_df.sample()

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,to2_off,to2_def,game_id,result_tie,result_win,wins,losses,ties
19304,nyj,2004,2004-09-12,nyj-2004,cin,200,1,Sun,W,1-0,31.0,24.0,24.0,438.0,219.0,219.0,1.0,19.0,351.0,238.0,113.0,2.0,1.0,NaN,20.0,-2.0,-8.0,1,1.0,4.0,cin-nyj-2004-09-12,0,1,1,0,NaN


Okay, almost there. Our ties are coming through, but we get NaNs for teams that haven't tied, so we'll need to replace those with zeros.

In [18]:
# fill ties NaNs with zero
game_df['ties'] = game_df.ties.fillna(0)

In [19]:
# add 'margin' col
margins = game_df.pts_off - game_df.pts_def
game_df.insert(loc=11, column='margin', value=margins)
game_df[mask].head()

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,to2_off,to2_def,game_id,result_tie,result_win,wins,losses,ties
9,crd,1960,1960-11-27,crd-1960,cle,196,10,Sun,T,5-4-1,17.0,0.0,17.0,21.0,322.0,127.0,195.0,6.0,17.0,274.0,88.0,186.0,2.0,NaN,NaN,NaN,NaN,NaN,1,36.0,4.0,cle-crd-1960-11-27,1,0,5,4,1
30,crd,1962,1962-10-14,crd-1962,was,196,5,Sun,T,1-3-1,17.0,0.0,17.0,16.0,309.0,251.0,58.0,2.0,15.0,321.0,259.0,62.0,1.0,NaN,NaN,NaN,NaN,NaN,1,4.0,1.0,crd-was-1962-10-14,1,0,1,3,1
55,crd,1964,1964-09-20,crd-1964,cle,196,2,Sun,T,1-0-1,33.0,0.0,33.0,17.0,344.0,230.0,114.0,3.0,18.0,281.0,166.0,115.0,3.0,NaN,NaN,NaN,NaN,NaN,0,9.0,9.0,cle-crd-1964-09-20,1,0,1,0,1
63,crd,1964,1964-11-15,crd-1964,nyg,196,10,Sun,T,5-3-2,10.0,0.0,10.0,12.0,143.0,105.0,38.0,3.0,12.0,212.0,131.0,81.0,3.0,NaN,NaN,NaN,NaN,NaN,1,9.0,9.0,crd-nyg-1964-11-15,1,0,5,3,2
87,crd,1966,1966-10-16,crd-1966,dal,196,6,Sun,T,5-0-1,10.0,0.0,10.0,13.0,175.0,130.0,45.0,NaN,17.0,326.0,191.0,135.0,4.0,NaN,NaN,NaN,NaN,NaN,1,NaN,16.0,crd-dal-1966-10-16,1,0,5,0,1


In [20]:
# drop rows for bye weeks
game_df.dropna(axis=0, how='any', subset=['game_outcome'], inplace=True)


# drop exp_pts cols
game_df.drop(['exp_pts_off', 'exp_pts_def', 'exp_pts_st'], axis=1, inplace=True)

In [21]:
# get integer week_num col

def apply_season_week_col(row):
    week_val = row['week_num']
    if week_val.isdigit():
        return int(week_val)
    else:
        return 0
    
game_df['season_week'] = game_df.apply(apply_season_week_col, axis=1)
game_df

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,home,to2_off,to2_def,game_id,result_tie,result_win,wins,losses,ties,season_week
0,crd,1960,1960-09-23,crd-1960,ram,196,1,Fri,W,1-0,43.0,22.0,21.0,20.0,430.0,281.0,149.0,5.0,16.0,238.0,188.0,50.0,6.0,NaN,NaN,0,25.0,36.0,crd-ram-1960-09-23,0,1,1,0,0,1
1,crd,1960,1960-10-02,crd-1960,nyg,196,2,Sun,L,1-1,14.0,-21.0,35.0,11.0,222.0,115.0,107.0,5.0,18.0,405.0,268.0,137.0,6.0,NaN,NaN,1,25.0,36.0,crd-nyg-1960-10-02,0,0,1,1,0,2
2,crd,1960,1960-10-09,crd-1960,phi,196,3,Sun,L,1-2,27.0,-4.0,31.0,20.0,364.0,145.0,219.0,6.0,14.0,293.0,199.0,94.0,3.0,NaN,NaN,0,36.0,9.0,crd-phi-1960-10-09,0,0,1,2,0,3
3,crd,1960,1960-10-16,crd-1960,pit,196,4,Sun,L,1-3,14.0,-13.0,27.0,20.0,295.0,154.0,141.0,6.0,12.0,237.0,146.0,91.0,NaN,NaN,NaN,0,36.0,NaN,crd-pit-1960-10-16,0,0,1,3,0,4
4,crd,1960,1960-10-23,crd-1960,dal,196,5,Sun,W,2-3,12.0,2.0,10.0,17.0,306.0,80.0,226.0,3.0,9.0,175.0,124.0,51.0,2.0,NaN,NaN,1,9.0,4.0,crd-dal-1960-10-23,0,1,2,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29393,was,2020,2020-12-13,was-2020,sfo,202,14,Sun,W,6-7,23.0,8.0,15.0,12.0,193.0,95.0,98.0,1.0,21.0,344.0,236.0,108.0,3.0,4.0,NaN,0,1.0,9.0,sfo-was-2020-12-13,0,1,6,7,0,14
29394,was,2020,2020-12-20,was-2020,sea,202,15,Sun,L,6-8,15.0,-5.0,20.0,26.0,353.0,269.0,84.0,2.0,16.0,302.0,121.0,181.0,1.0,1.0,NaN,1,4.0,1.0,sea-was-2020-12-20,0,0,6,8,0,15
29395,was,2020,2020-12-27,was-2020,car,202,16,Sun,L,6-9,13.0,-7.0,20.0,20.0,386.0,278.0,108.0,4.0,19.0,280.0,167.0,113.0,2.0,4.0,NaN,1,16.0,4.0,car-was-2020-12-27,0,0,6,9,0,16
29396,was,2020,2020-01-03,was-2020,phi,202,17,Sun,W,7-9,20.0,6.0,14.0,16.0,248.0,145.0,103.0,2.0,16.0,216.0,98.0,118.0,3.0,8.0,NaN,0,4.0,9.0,phi-was-2020-01-03,0,1,7,9,0,17


In [26]:
# add 'playoff_game' column
game_df['playoff_game'] = game_df['season_week'] == 0
game_df

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,home,to2_off,to2_def,game_id,result_tie,result_win,wins,losses,ties,season_week,prev_wins,prev_losses,prev_season_week,prev_ties,prev_pts_off,prev_pts_def,prev_margin,prev_first_down_off,prev_yards_off,prev_pass_yds_off,prev_rush_yds_off,prev_to_off,prev_first_down_def,prev_yards_def,prev_pass_yds_def,prev_rush_yds_def,prev_to_def,prev_result_tie,prev_result_win,off_bye,playoff_game
0,crd,1960,1960-09-23,crd-1960,ram,196,1,Fri,W,1-0,43.0,22.0,21.0,20.0,430.0,281.0,149.0,5.0,16.0,238.0,188.0,50.0,6.0,NaN,NaN,0,25.0,36.0,crd-ram-1960-09-23,0,1,1,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False
1,crd,1960,1960-10-02,crd-1960,nyg,196,2,Sun,L,1-1,14.0,-21.0,35.0,11.0,222.0,115.0,107.0,5.0,18.0,405.0,268.0,137.0,6.0,NaN,NaN,1,25.0,36.0,crd-nyg-1960-10-02,0,0,1,1,0,2,1,0,1.0,0,43.0,21.0,22.0,20.0,430.0,281.0,149.0,5.0,16.0,238.0,188.0,50.0,6.0,0.0,1.0,0,False
2,crd,1960,1960-10-09,crd-1960,phi,196,3,Sun,L,1-2,27.0,-4.0,31.0,20.0,364.0,145.0,219.0,6.0,14.0,293.0,199.0,94.0,3.0,NaN,NaN,0,36.0,9.0,crd-phi-1960-10-09,0,0,1,2,0,3,1,1,2.0,0,14.0,35.0,-21.0,11.0,222.0,115.0,107.0,5.0,18.0,405.0,268.0,137.0,6.0,0.0,0.0,0,False
3,crd,1960,1960-10-16,crd-1960,pit,196,4,Sun,L,1-3,14.0,-13.0,27.0,20.0,295.0,154.0,141.0,6.0,12.0,237.0,146.0,91.0,NaN,NaN,NaN,0,36.0,NaN,crd-pit-1960-10-16,0,0,1,3,0,4,1,2,3.0,0,27.0,31.0,-4.0,20.0,364.0,145.0,219.0,6.0,14.0,293.0,199.0,94.0,3.0,0.0,0.0,0,False
4,crd,1960,1960-10-23,crd-1960,dal,196,5,Sun,W,2-3,12.0,2.0,10.0,17.0,306.0,80.0,226.0,3.0,9.0,175.0,124.0,51.0,2.0,NaN,NaN,1,9.0,4.0,crd-dal-1960-10-23,0,1,2,3,0,5,1,3,4.0,0,14.0,27.0,-13.0,20.0,295.0,154.0,141.0,6.0,12.0,237.0,146.0,91.0,NaN,0.0,0.0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29393,was,2020,2020-12-13,was-2020,sfo,202,14,Sun,W,6-7,23.0,8.0,15.0,12.0,193.0,95.0,98.0,1.0,21.0,344.0,236.0,108.0,3.0,4.0,NaN,0,1.0,9.0,sfo-was-2020-12-13,0,1,6,7,0,14,5,7,13.0,0,23.0,17.0,6.0,18.0,318.0,273.0,45.0,NaN,17.0,326.0,305.0,21.0,1.0,0.0,1.0,0,False
29394,was,2020,2020-12-20,was-2020,sea,202,15,Sun,L,6-8,15.0,-5.0,20.0,26.0,353.0,269.0,84.0,2.0,16.0,302.0,121.0,181.0,1.0,1.0,NaN,1,4.0,1.0,sea-was-2020-12-20,0,0,6,8,0,15,6,7,14.0,0,23.0,15.0,8.0,12.0,193.0,95.0,98.0,1.0,21.0,344.0,236.0,108.0,3.0,0.0,1.0,0,False
29395,was,2020,2020-12-27,was-2020,car,202,16,Sun,L,6-9,13.0,-7.0,20.0,20.0,386.0,278.0,108.0,4.0,19.0,280.0,167.0,113.0,2.0,4.0,NaN,1,16.0,4.0,car-was-2020-12-27,0,0,6,9,0,16,6,8,15.0,0,15.0,20.0,-5.0,26.0,353.0,269.0,84.0,2.0,16.0,302.0,121.0,181.0,1.0,0.0,0.0,0,False
29396,was,2020,2020-01-03,was-2020,phi,202,17,Sun,W,7-9,20.0,6.0,14.0,16.0,248.0,145.0,103.0,2.0,16.0,216.0,98.0,118.0,3.0,8.0,NaN,0,4.0,9.0,phi-was-2020-01-03,0,1,7,9,0,17,6,9,16.0,0,13.0,20.0,-7.0,20.0,386.0,278.0,108.0,4.0,19.0,280.0,167.0,113.0,2.0,0.0,0.0,0,False


In [22]:
# add 'prev_week' cols

cols_to_shift = ['wins', 'losses', 'season_week', 'ties', 'pts_off', 'pts_def',
       'margin', 'first_down_off', 'yards_off', 'pass_yds_off', 'rush_yds_off',
       'to_off', 'first_down_def', 'yards_def', 'pass_yds_def', 'rush_yds_def',
       'to_def', 'result_tie', 'result_win']

for col in cols_to_shift:
    new_col = 'prev_' + col
    game_df[new_col] = game_df.groupby('team_year')[col].apply(lambda grp: grp.shift(1))

In [24]:
# add 'off_bye' (boolean) col
    
def apply_off_bye(row):
    if row['season_week'] > 1: # playoff games have week_num = 0
        off_bye = row['season_week'] - row['prev_season_week'] == 2
    else:
        off_bye = False
    if off_bye == True:
        return 1
    return 0


game_df['off_bye'] = game_df[['season_week', 'prev_season_week']].apply(apply_off_bye, axis=1)



In [25]:
mask = game_df.team_year == 'chi-2019'
game_df[mask]

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,home,to2_off,to2_def,game_id,result_tie,result_win,wins,losses,ties,season_week,prev_wins,prev_losses,prev_season_week,prev_ties,prev_pts_off,prev_pts_def,prev_margin,prev_first_down_off,prev_yards_off,prev_pass_yds_off,prev_rush_yds_off,prev_to_off,prev_first_down_def,prev_yards_def,prev_pass_yds_def,prev_rush_yds_def,prev_to_def,prev_result_tie,prev_result_win,off_bye
4819,chi,2019,2019-09-05,chi-2019,gnb,201,1,Thu,L,0-1,3.0,-7.0,10.0,16.0,254.0,208.0,46.0,1.0,13.0,213.0,166.0,47.0,NaN,8.0,NaN,1,1.0,NaN,chi-gnb-2019-09-05,0,0,0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4820,chi,2019,2019-09-15,chi-2019,den,201,2,Sun,W,1-1,16.0,2.0,14.0,18.0,273.0,120.0,153.0,NaN,27.0,372.0,282.0,90.0,1.0,4.0,NaN,0,NaN,1.0,chi-den-2019-09-15,0,1,1,1,0,2,0,1,1.0,0,3.0,10.0,-7.0,16.0,254.0,208.0,46.0,1.0,13.0,213.0,166.0,47.0,NaN,0.0,0.0,0
4821,chi,2019,2019-09-23,chi-2019,was,201,3,Mon,W,2-1,31.0,16.0,15.0,21.0,298.0,208.0,90.0,1.0,25.0,356.0,287.0,69.0,5.0,8.0,NaN,0,1.0,25.0,chi-was-2019-09-23,0,1,2,1,0,3,1,1,2.0,0,16.0,14.0,2.0,18.0,273.0,120.0,153.0,NaN,27.0,372.0,282.0,90.0,1.0,0.0,1.0,0
4822,chi,2019,2019-09-29,chi-2019,min,201,4,Sun,W,3-1,16.0,10.0,6.0,17.0,269.0,197.0,72.0,NaN,15.0,222.0,182.0,40.0,2.0,4.0,NaN,1,NaN,4.0,chi-min-2019-09-29,0,1,3,1,0,4,2,1,3.0,0,31.0,15.0,16.0,21.0,298.0,208.0,90.0,1.0,25.0,356.0,287.0,69.0,5.0,0.0,1.0,0
4823,chi,2019,2019-10-06,chi-2019,rai,201,5,Sun,L,3-2,21.0,-3.0,24.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,1.0,NaN,0,4.0,4.0,chi-rai-2019-10-06,0,0,3,2,0,5,3,1,4.0,0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,NaN,15.0,222.0,182.0,40.0,2.0,0.0,1.0,0
4825,chi,2019,2019-10-20,chi-2019,nor,201,7,Sun,L,3-3,25.0,-11.0,36.0,16.0,252.0,235.0,17.0,2.0,24.0,424.0,273.0,151.0,NaN,4.0,NaN,1,4.0,NaN,chi-nor-2019-10-20,0,0,3,3,0,7,3,2,5.0,0,21.0,24.0,-3.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,0.0,0.0,1
4826,chi,2019,2019-10-27,chi-2019,sdg,201,8,Sun,L,3-4,16.0,-1.0,17.0,26.0,388.0,226.0,162.0,2.0,11.0,231.0,195.0,36.0,1.0,1.0,NaN,1,4.0,1.0,chi-sdg-2019-10-27,0,0,3,4,0,8,3,3,7.0,0,25.0,36.0,-11.0,16.0,252.0,235.0,17.0,2.0,24.0,424.0,273.0,151.0,NaN,0.0,0.0,0
4827,chi,2019,2019-11-03,chi-2019,phi,201,9,Sun,L,3-5,14.0,-8.0,22.0,10.0,164.0,102.0,62.0,1.0,26.0,373.0,227.0,146.0,NaN,1.0,NaN,0,1.0,NaN,chi-phi-2019-11-03,0,0,3,5,0,9,3,4,8.0,0,16.0,17.0,-1.0,26.0,388.0,226.0,162.0,2.0,11.0,231.0,195.0,36.0,1.0,0.0,0.0,0
4828,chi,2019,2019-11-10,chi-2019,det,201,10,Sun,W,4-5,20.0,7.0,13.0,13.0,226.0,145.0,81.0,NaN,21.0,357.0,259.0,98.0,1.0,1.0,NaN,1,NaN,1.0,chi-det-2019-11-10,0,1,4,5,0,10,3,5,9.0,0,14.0,22.0,-8.0,10.0,164.0,102.0,62.0,1.0,26.0,373.0,227.0,146.0,NaN,0.0,0.0,0
4829,chi,2019,2019-11-17,chi-2019,ram,201,11,Sun,L,4-6,7.0,-10.0,17.0,17.0,267.0,193.0,74.0,1.0,13.0,283.0,173.0,110.0,2.0,8.0,NaN,0,1.0,4.0,chi-ram-2019-11-17,0,0,4,6,0,11,4,5,10.0,0,20.0,13.0,7.0,13.0,226.0,145.0,81.0,NaN,21.0,357.0,259.0,98.0,1.0,0.0,1.0,0


In [ ]:
game_df.columns

In [ ]:
# create rolling three feature for these stats:
# using margin instead of wins here... if we want to get num wins in last three weeks,
#  will need to do differently using sum instead of mean

# don't use prev_ columns, we will just shift them down when we use .transform()
roll_cols = ['result_win', 'result_tie', 'pts_off', 'pts_def', 'margin', 'first_down_off', 'yards_off', 
             'pass_yds_off', 'rush_yds_off', 'to_off', 'to2_off', 'yards_def', 'pass_yds_def',
             'rush_yds_def', 'to_def', 'to2_def']

In [ ]:
roll3_cols = ['roll3_' + col_name for col_name in roll_cols]

game_df[roll3_cols] = (game_df.groupby('team_year')[roll_cols]
                      .transform(lambda x: round(x.shift(1).rolling(3).mean(), 3)))


In [ ]:
# get rolling wins (sum) (commented out to try using mean)

rolling_wins = (game_df.groupby('team_year')['result_win']
                      .transform(lambda x: round(x.shift(1).rolling(3).sum(), 3)))

game_df.insert(loc=53, column='roll3_num_wins', value=rolling_wins)

rolling_ties = (game_df.groupby('team_year')['result_win']
                      .transform(lambda x: round(x.shift(1).rolling(3).sum(), 3)))

game_df.insert(loc=53, column='roll3_num_ties', value=rolling_ties)

Now let's add more columns with **Expontentially Weighted Moving Averages (EWMA)** of our stats. This is similar to taking a rolling mean of a team's previous 6 performances, but EWMA weights the most recent game in the window much higher than the week before that, and so on.

For this, we will use a 3-16 week span (greedy). My hypothesis is that the most recent performances are the best predictors, and this should help further quantify recent performance.

In [ ]:
# add ewma cols

ewma_cols = ['ewma_' + col_name for col_name in roll_cols]

game_df[ewma_cols] = (game_df.groupby('team_year')[roll_cols]
                      .transform(lambda x: round(x.shift(1).ewm(span=16, min_periods=3).mean(), 3)))


In [ ]:
mask = game_df.team_year == 'nor-2014'
game_df[mask][['week_num', 'team', 'pass_yds_off', 'prev_pass_yds_off', 'ewma_pass_yds_off', 'roll3_pass_yds_off']].head(18)

In [ ]:
np.mean([197, 120, 208])

Now, let's delete the first three rows of each team-year. We have to do this because we will be using at least some rolling columns for our model.

In [ ]:
game_df.dropna(axis=0, how="any", subset=["roll3_pts_off"], inplace=True)

Looks good!!

**Now, we have to solve a pretty complex problem.**

Currently, each row has all the stats needed for the team in the 'team' column. However, we don't have the same information for the opponent in the same row. Let's make that happen.

To further clarify, for each row, we need to pull the opposing team's rolling stats, adding them back into the same row. This way, our model will only need one row to make a prediction.

Let's take a look at an example.

In [ ]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')
game_df[mask]


So, in the above table, we need to get all of the roll3_x and ewma_x columns from the adjacent tables, respectively. We'll prefix these with opp_.

In [ ]:
game_df.columns

In [ ]:
opp_pull_cols = ['game_id', 'team', 'opp', 'prev_wins', 'prev_losses',
       'prev_ties', 'roll3_result_win', 'roll3_num_wins', 'roll3_result_tie', 
       'roll3_num_ties', 'prev_pts_off', 'prev_pts_def',
       'prev_margin', 'prev_first_down_off', 'prev_yards_off',
       'prev_pass_yds_off', 'prev_rush_yds_off', 'prev_to_off',
       'prev_first_down_def', 'prev_yards_def', 'prev_pass_yds_def',
       'prev_rush_yds_def', 'prev_to_def', 'prev_result_tie',
       'prev_result_win', 'roll3_pts_off', 'roll3_pts_def', 'roll3_margin',
       'roll3_first_down_off', 'roll3_yards_off', 'roll3_pass_yds_off',
       'roll3_rush_yds_off', 'roll3_to_off', 'roll3_yards_def',
       'roll3_pass_yds_def', 'roll3_rush_yds_def', 'roll3_to_def', 'ewma_result_win',
       'ewma_result_tie', 'ewma_num_wins', 'ewma_num_ties', 
       'ewma_pts_off', 'ewma_pts_def', 'ewma_margin', 'ewma_first_down_off',
       'ewma_yards_off', 'ewma_pass_yds_off', 'ewma_rush_yds_off',
       'ewma_to_off', 'ewma_yards_def', 'ewma_pass_yds_def',
       'ewma_rush_yds_def', 'ewma_to_def']

In [ ]:
# convert all cols to float

game_df[opp_pull_cols] = game_df[opp_pull_cols].fillna(0)
for col in opp_pull_cols[3:]:
    game_df[col] = game_df[col].astype(float)

**Let's solve this using `pd.merge`, using a self-join.**

In [ ]:
game_df = (game_df
             .merge(right=game_df[opp_pull_cols],
                    left_on=['game_id', 'team'],
                    right_on=['game_id', 'opp'],
                    suffixes=[None, '_opp']))

Let's make sure it worked by checking a couple of random game_ids.

In [ ]:
mask = (game_df.game_id == 'dal-nyg-2019-11-04')

game_df[mask][['game_id', 'team', 'opp', 'prev_rush_yds_off',
                 'prev_rush_yds_off_opp', 'roll3_margin', 'roll3_margin_opp']]

In [ ]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')
game_df[mask][['game_id', 'team', 'opp', 'prev_rush_yds_off',
                 'prev_rush_yds_off_opp', 'roll3_margin', 'roll3_margin_opp']]

In [ ]:
mask = (game_df.game_id == 'chi-gnb-1960-12-04')
game_df[mask][['game_id', 'team', 'opp', 'prev_rush_yds_off',
                 'prev_rush_yds_off_opp', 'roll3_margin', 'roll3_margin_opp']]

## Now let's take another look to make sure everything looks good.

If it does, we're ready to start modeling!

In [ ]:
mask = game_df['team_year'] == 'chi-2019'
game_df[mask]

Looks great. Now it's time to start modeling our data! Head over to `model_exploration.ipynb` to continue following along.